## 기초 탐색 및 데이터 준비

#### 학습 데이터 불러오기

In [1]:
import pandas as pd
import os

In [2]:
# df의 시간 범위: 2016-06-01 ~ 2019-02-28
df = pd.read_csv("funda_train.csv")
df.head()

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000


In [3]:
# submission_df의 범위: 2019-03-01 ~ 2019-05-31
submission_df = pd.read_csv("submission.csv")
submission_df.head()

,store_id,amount
0,0,0
1,1,0
2,2,0
3,4,0
4,5,0


#### 변수 목록 탐색

In [4]:
# 상점 ID는 일치
print(df['store_id'].unique())
print(submission_df['store_id'].unique())

[   0    1    2 ... 2134 2135 2136]
[   0    1    2 ... 2134 2135 2136]


In [5]:
print(df.columns)
print(submission_df.columns)

# 일치하는 컬럼이 store_id와 amount 밖에 없으므로, 새로운 특징을 추출하는 것이 바람직함

Index(['store_id', 'card_id', 'card_company', 'transacted_date',
       'transacted_time', 'installment_term', 'region', 'type_of_business',
       'amount'],
      dtype='object')
Index(['store_id', 'amount'], dtype='object')


- 예측 대상은 3개월 합계이고, 가지고 있는 데이터는 분단위로 정리되어 있음
- t-2, t-1, t월의 데이터로 t + 1, t + 2, t + 3월의 매출 합계를 예측하는 것으로 문제를 정의
- 따라서 거래 내역을 요약하여 월별로 데이터를 새로 정의하는 것이 중요

## 학습 데이터 구축

#### 년/월 추출

In [6]:
# .str.split을 이용한 년/월 추출
df['transacted_year'] = df['transacted_date'].str.split('-', expand = True).iloc[:, 0].astype(int)
df['transacted_month'] = df['transacted_date'].str.split('-', expand = True).iloc[:, 1].astype(int)
df.head()

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,transacted_year,transacted_month
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857,2016,6
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857,2016,6
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000,2016,6
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857,2016,6
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000,2016,6


In [7]:
# 데이터 병합을 위한 새로운 컬럼 생성 및 기존 시간 변수 삭제
df['t'] = (df['transacted_year'] - 2016) * 12 + df['transacted_month']
df.drop(['transacted_year', 'transacted_month', 'transacted_date', 'transacted_time'], axis = 1, inplace = True)

#### 불필요한 변수 제거
- card_id, card_company는 특징으로 사용하기에는 너무 세분화될 수 있을 뿐만 아니라, 특징으로 유효할 가능성이 없다고 판단하여 삭제

In [8]:
df.drop(['card_id', 'card_company'], axis = 1, inplace = True)

#### 업종 특성, 지역, 할부 평균 탐색

In [9]:
df['installment_term'].value_counts().head() # 대부분이 일시불이므로, installment_term 변수를 할부인지 아닌지를 여부로 변환

0    6327632
3     134709
2      42101
5      23751
6      10792
Name: installment_term, dtype: int64

In [10]:
df['installment_term'] = (df['installment_term'] > 0).astype(int)
df['installment_term'].value_counts()

0    6327632
1     228981
Name: installment_term, dtype: int64

In [11]:
# 상점별 평균 할부 비율
installment_term_per_store = df.groupby(['store_id'])['installment_term'].mean()
installment_term_per_store.head()

store_id
0    0.038384
1    0.000000
2    0.083904
4    0.001201
5    0.075077
Name: installment_term, dtype: float64

In [12]:
# groupby에 결측을 포함시키기 위해, 결측을 문자로 대체
# 지역은 너무 많아서 그대로 활용하기 어려움. 따라서 그대로 더미화하지 않고, 이를 기반으로 한 새로운 변수를 파생해서 사용
df['region'].fillna('없음', inplace = True)
df['region'].value_counts().head()

없음        2042766
경기 수원시     122029
충북 청주시     116766
경남 창원시     107147
경남 김해시     100673
Name: region, dtype: int64

In [13]:
df['type_of_business'].value_counts().head()
# 업종도 그 수가 너무 많아 그대로 활용하기 어려움

한식 음식점업    745905
두발 미용업     178475
의복 소매업     158234
기타 주점업     102413
치킨 전문점      89277
Name: type_of_business, dtype: int64

In [14]:
# groupby에 결측을 포함시키기 위해, 결측을 문자로 대체
df['type_of_business'].fillna('없음', inplace = True)

#### 학습 데이터 구조 작성

In [15]:
# 'store_id', 'region', 'type_of_business', 't'를 기준으로 중복을 제거한 뒤, 해당 컬럼만 가져옴
train_df = df.drop_duplicates(subset = ['store_id', 'region', 'type_of_business', 't'])[['store_id', 'region', 'type_of_business', 't']]
train_df.head()

,store_id,region,type_of_business,t
0,0,없음,기타 미용업,6
145,0,없음,기타 미용업,7
323,0,없음,기타 미용업,8
494,0,없음,기타 미용업,9
654,0,없음,기타 미용업,10


#### 평균 할부율 부착

In [16]:
train_df['평균할부율'] = train_df['store_id'].replace(installment_term_per_store.to_dict())

#### t-1, t-2, t-3 시점의 매출 합계 부착

In [17]:
# store_id와 t에 따른 amount 합계 계산: amount_sum_per_t_and_sid
amount_sum_per_t_and_sid = df.groupby(['store_id', 't'], as_index = False)['amount'].sum()
amount_sum_per_t_and_sid.head()

,store_id,t,amount
0,0,6,7.470000e+05
1,0,7,1.005000e+06
2,0,8,8.715714e+05
3,0,9,8.978571e+05
4,0,10,8.354286e+05


In [18]:
# 몇몇 상점은 중간이 비어있음을 확인 => merge에서 문제가 생길 수 있음
amount_sum_per_t_and_sid.groupby(['store_id'])['t'].count().head(10)

store_id
0     33
1     33
2     33
4     33
5     33
6     31
7     31
8     28
9     29
10    23
Name: t, dtype: int64

In [19]:
# 따라서 모든 값을 채우기 위해, 피벗 테이블을 생성하고 결측을 바로 앞 값으로 채움
amount_sum_per_t_and_sid = pd.pivot_table(df, values = 'amount', index = 'store_id', columns = 't', aggfunc = 'sum')
amount_sum_per_t_and_sid.head(10)

t,6,7,8,9,10,11,12,13,14,15,...,29,30,31,32,33,34,35,36,37,38
store_id,,,,,,,,,,,,,,,,,,,,,
0,747000.000000,1.005000e+06,871571.428571,8.978571e+05,8.354286e+05,6.970000e+05,7.618571e+05,585642.857143,7.940000e+05,720257.142857,...,6.864286e+05,7.072857e+05,7.587143e+05,6.798571e+05,6.518571e+05,7.390000e+05,6.760000e+05,8.745714e+05,6.828571e+05,5.152857e+05
1,137214.285714,1.630000e+05,118142.857143,9.042857e+04,1.180714e+05,1.118571e+05,1.155714e+05,129642.857143,1.602143e+05,168428.571429,...,8.050000e+04,7.828571e+04,1.007857e+05,9.214286e+04,6.357143e+04,9.500000e+04,8.078571e+04,8.528571e+04,1.482857e+05,7.742857e+04
2,260714.285714,8.285714e+04,131428.571429,1.428571e+05,1.097143e+05,1.985714e+05,1.600000e+05,180714.285714,1.542857e+05,43571.428571,...,4.728571e+05,3.542857e+05,6.892857e+05,4.578571e+05,4.807143e+05,5.100000e+05,1.854286e+05,3.407143e+05,4.078571e+05,4.968571e+05
4,733428.571429,7.689286e+05,698428.571429,9.364286e+05,7.627143e+05,8.595714e+05,1.069857e+06,689142.857143,1.050143e+06,970285.714286,...,7.754286e+05,8.812857e+05,1.050929e+06,8.492857e+05,6.981429e+05,8.284286e+05,8.830000e+05,9.238571e+05,9.448571e+05,8.822857e+05
5,342500.000000,4.327143e+05,263500.000000,2.321429e+05,2.115714e+05,1.820857e+05,1.475714e+05,120957.142857,1.864286e+05,169000.000000,...,4.438571e+05,5.637143e+05,6.070714e+05,4.828857e+05,1.950000e+05,3.249286e+05,3.833000e+05,3.995714e+05,3.230000e+05,2.155143e+05
6,NaN,NaN,568857.142857,1.440143e+06,1.238857e+06,1.055429e+06,9.268571e+05,885642.857143,8.003571e+05,930714.285714,...,1.808357e+06,1.752286e+06,1.583786e+06,1.628786e+06,2.074071e+06,1.907643e+06,2.389143e+06,2.230286e+06,2.015500e+06,2.463857e+06
7,NaN,NaN,107857.142857,3.756429e+05,3.236429e+05,3.450000e+05,2.914286e+05,231614.285714,2.713571e+05,249857.142857,...,2.657143e+05,4.195429e+05,4.628429e+05,4.231286e+05,3.203286e+05,4.200286e+05,3.143857e+05,3.024143e+05,1.364714e+05,5.797143e+04
8,NaN,NaN,NaN,NaN,NaN,1.925714e+05,7.355000e+05,467857.142857,4.756429e+05,603500.000000,...,1.837429e+06,1.359857e+06,1.213543e+06,1.086000e+06,1.369557e+06,1.272071e+06,1.260557e+06,1.157257e+06,1.134671e+06,1.298329e+06
9,NaN,NaN,NaN,NaN,1.071429e+05,6.371429e+05,6.035714e+05,225428.571429,2.871429e+05,344428.571429,...,6.385714e+05,2.765714e+05,3.400000e+05,2.542857e+05,9.265714e+05,8.714286e+05,6.928571e+05,6.628571e+05,3.700000e+05,4.057143e+05


In [20]:
# 따라서 모든 값을 채우기 위해, 피벗 테이블을 생성하고 결측을 바로 앞 값으로 채움
amount_sum_per_t_and_sid = amount_sum_per_t_and_sid.fillna(method = 'ffill', axis = 1).fillna(method = 'bfill', axis = 1)
amount_sum_per_t_and_sid.head(10)

t,6,7,8,9,10,11,12,13,14,15,...,29,30,31,32,33,34,35,36,37,38
store_id,,,,,,,,,,,,,,,,,,,,,
0,747000.000000,1.005000e+06,871571.428571,8.978571e+05,8.354286e+05,6.970000e+05,7.618571e+05,585642.857143,7.940000e+05,720257.142857,...,6.864286e+05,7.072857e+05,7.587143e+05,6.798571e+05,6.518571e+05,7.390000e+05,6.760000e+05,8.745714e+05,6.828571e+05,5.152857e+05
1,137214.285714,1.630000e+05,118142.857143,9.042857e+04,1.180714e+05,1.118571e+05,1.155714e+05,129642.857143,1.602143e+05,168428.571429,...,8.050000e+04,7.828571e+04,1.007857e+05,9.214286e+04,6.357143e+04,9.500000e+04,8.078571e+04,8.528571e+04,1.482857e+05,7.742857e+04
2,260714.285714,8.285714e+04,131428.571429,1.428571e+05,1.097143e+05,1.985714e+05,1.600000e+05,180714.285714,1.542857e+05,43571.428571,...,4.728571e+05,3.542857e+05,6.892857e+05,4.578571e+05,4.807143e+05,5.100000e+05,1.854286e+05,3.407143e+05,4.078571e+05,4.968571e+05
4,733428.571429,7.689286e+05,698428.571429,9.364286e+05,7.627143e+05,8.595714e+05,1.069857e+06,689142.857143,1.050143e+06,970285.714286,...,7.754286e+05,8.812857e+05,1.050929e+06,8.492857e+05,6.981429e+05,8.284286e+05,8.830000e+05,9.238571e+05,9.448571e+05,8.822857e+05
5,342500.000000,4.327143e+05,263500.000000,2.321429e+05,2.115714e+05,1.820857e+05,1.475714e+05,120957.142857,1.864286e+05,169000.000000,...,4.438571e+05,5.637143e+05,6.070714e+05,4.828857e+05,1.950000e+05,3.249286e+05,3.833000e+05,3.995714e+05,3.230000e+05,2.155143e+05
6,568857.142857,5.688571e+05,568857.142857,1.440143e+06,1.238857e+06,1.055429e+06,9.268571e+05,885642.857143,8.003571e+05,930714.285714,...,1.808357e+06,1.752286e+06,1.583786e+06,1.628786e+06,2.074071e+06,1.907643e+06,2.389143e+06,2.230286e+06,2.015500e+06,2.463857e+06
7,107857.142857,1.078571e+05,107857.142857,3.756429e+05,3.236429e+05,3.450000e+05,2.914286e+05,231614.285714,2.713571e+05,249857.142857,...,2.657143e+05,4.195429e+05,4.628429e+05,4.231286e+05,3.203286e+05,4.200286e+05,3.143857e+05,3.024143e+05,1.364714e+05,5.797143e+04
8,192571.428571,1.925714e+05,192571.428571,1.925714e+05,1.925714e+05,1.925714e+05,7.355000e+05,467857.142857,4.756429e+05,603500.000000,...,1.837429e+06,1.359857e+06,1.213543e+06,1.086000e+06,1.369557e+06,1.272071e+06,1.260557e+06,1.157257e+06,1.134671e+06,1.298329e+06
9,107142.857143,1.071429e+05,107142.857143,1.071429e+05,1.071429e+05,6.371429e+05,6.035714e+05,225428.571429,2.871429e+05,344428.571429,...,6.385714e+05,2.765714e+05,3.400000e+05,2.542857e+05,9.265714e+05,8.714286e+05,6.928571e+05,6.628571e+05,3.700000e+05,4.057143e+05


In [21]:
# stack을 이용하여, 컬럼도 행 인덱스로 밀어넣고, 인덱스를 초기화하여 인덱스를 컬럼으로 가져옴
amount_sum_per_t_and_sid = amount_sum_per_t_and_sid.stack().reset_index()
amount_sum_per_t_and_sid.rename({0:"amount"}, axis = 1, inplace = True)

In [22]:
# t - k (k = 1, 2, 3) 시점의 부착
# train_df의 t는 amount_sum_per_t_and_sid의 t-k과 부착되어야 하므로, amount_sum_per_t_and_sid의 t에 k를 더함

for k in range(1, 4):
    amount_sum_per_t_and_sid['t_{}'.format(k)] = amount_sum_per_t_and_sid['t'] + k
    train_df = pd.merge(train_df, amount_sum_per_t_and_sid.drop('t', axis = 1), left_on = ['store_id', 't'], right_on = ['store_id', 't_{}'.format(k)])
    
    # 부착한 뒤, 불필요한 변수 제거 및 변수명 변경: 다음 이터레이션에서의 병합이 잘되게 하기 위해서
    train_df.rename({"amount":"{}_before_amount".format(k)}, axis = 1, inplace = True)
    train_df.drop(['t_{}'.format(k)], axis = 1, inplace = True)
    amount_sum_per_t_and_sid.drop(['t_{}'.format(k)], axis = 1, inplace = True)
    
train_df.head()

,store_id,region,type_of_business,t,평균할부율,1_before_amount,2_before_amount,3_before_amount
0,0,없음,기타 미용업,9,0.038384,871571.428571,1.005000e+06,7.470000e+05
1,0,없음,기타 미용업,10,0.038384,897857.142857,8.715714e+05,1.005000e+06
2,0,없음,기타 미용업,11,0.038384,835428.571429,8.978571e+05,8.715714e+05
3,0,없음,기타 미용업,12,0.038384,697000.000000,8.354286e+05,8.978571e+05
4,0,없음,기타 미용업,13,0.038384,761857.142857,6.970000e+05,8.354286e+05


#### t-1, t-2, t-3 시점의 지역별 매출 합계 평균 부착

In [23]:
# amount_sum_per_t_and_sid의 store_id를 region으로 대체시키기
store_to_region = df[['store_id', 'region']].drop_duplicates().set_index(['store_id'])['region'].to_dict()
amount_sum_per_t_and_sid['region'] = amount_sum_per_t_and_sid['store_id'].replace(store_to_region)

# 지역별 평균 매출 계산
amount_mean_per_t_and_region = amount_sum_per_t_and_sid.groupby(['region', 't'], as_index = False)['amount'].mean()

In [24]:
# t - k (k = 1, 2, 3) 시점의 부착

for k in range(1, 4):
    amount_mean_per_t_and_region['t_{}'.format(k)] = amount_mean_per_t_and_region['t'] + k
    train_df = pd.merge(train_df, amount_mean_per_t_and_region.drop('t', axis = 1), left_on = ['region', 't'], right_on = ['region', 't_{}'.format(k)])
    train_df.rename({"amount":"{}_before_amount_of_region".format(k)}, axis = 1, inplace = True)
    
    train_df.drop(['t_{}'.format(k)], axis = 1, inplace = True)
    amount_mean_per_t_and_region.drop(['t_{}'.format(k)], axis = 1, inplace = True)    
    
train_df.head()

,store_id,region,type_of_business,t,평균할부율,1_before_amount,2_before_amount,3_before_amount,1_before_amount_of_region,2_before_amount_of_region,3_before_amount_of_region
0,0,없음,기타 미용업,9,0.038384,871571.428571,1.005000e+06,747000.000000,761987.421532,756108.674948,739654.068323
1,1,없음,없음,9,0.000000,118142.857143,1.630000e+05,137214.285714,761987.421532,756108.674948,739654.068323
2,2,없음,없음,9,0.083904,131428.571429,8.285714e+04,260714.285714,761987.421532,756108.674948,739654.068323
3,5,없음,의복 액세서리 및 모조 장신구 도매업,9,0.075077,263500.000000,4.327143e+05,342500.000000,761987.421532,756108.674948,739654.068323
4,7,없음,없음,9,0.011558,107857.142857,1.078571e+05,107857.142857,761987.421532,756108.674948,739654.068323


#### t-1, t-2, t-3 시점의 업종별 매출 합계 평균 부착

In [25]:
# amount_sum_per_t_and_sid의 store_id를 type_of_business으로 대체시키기
store_to_type_of_business = df[['store_id', 'type_of_business']].drop_duplicates().set_index(['store_id'])['type_of_business'].to_dict()
amount_sum_per_t_and_sid['type_of_business'] = amount_sum_per_t_and_sid['store_id'].replace(store_to_type_of_business)

# 지역별 평균 매출 계산
amount_mean_per_t_and_type_of_business = amount_sum_per_t_and_sid.groupby(['type_of_business', 't'], as_index = False)['amount'].mean()

In [26]:
# t - k (k = 1, 2, 3) 시점의 부착
# train_df의 t는 amount_sum_per_t_and_sid의 t-k과 부착되어야 하므로, amount_sum_per_t_and_sid의 t에 k를 더함

for k in range(1, 4):
    amount_mean_per_t_and_type_of_business['t_{}'.format(k)] = amount_mean_per_t_and_type_of_business['t'] + k
    train_df = pd.merge(train_df, amount_mean_per_t_and_type_of_business.drop('t', axis = 1), left_on = ['type_of_business', 't'], right_on = ['type_of_business', 't_{}'.format(k)])
    train_df.rename({"amount":"{}_before_amount_of_type_of_business".format(k)}, axis = 1, inplace = True)
    
    train_df.drop(['t_{}'.format(k)], axis = 1, inplace = True)
    amount_mean_per_t_and_type_of_business.drop(['t_{}'.format(k)], axis = 1, inplace = True)       
    
train_df.head()

,store_id,region,type_of_business,t,평균할부율,1_before_amount,2_before_amount,3_before_amount,1_before_amount_of_region,2_before_amount_of_region,3_before_amount_of_region,1_before_amount_of_type_of_business,2_before_amount_of_type_of_business,3_before_amount_of_type_of_business
0,0,없음,기타 미용업,9,0.038384,871571.428571,1.005000e+06,747000.000000,7.619874e+05,7.561087e+05,7.396541e+05,761025.0,804979.761905,679950.0
1,792,없음,기타 미용업,9,0.218887,681142.857143,8.808571e+05,733714.285714,7.619874e+05,7.561087e+05,7.396541e+05,761025.0,804979.761905,679950.0
2,23,경기 안양시,기타 미용업,9,0.048795,879242.857143,7.308571e+05,845285.714286,8.288317e+05,5.887330e+05,9.559733e+05,761025.0,804979.761905,679950.0
3,192,경기 화성시,기타 미용업,9,0.100542,579000.000000,5.234286e+05,551142.857143,1.234460e+06,1.227921e+06,1.180455e+06,761025.0,804979.761905,679950.0
4,536,서울 광진구,기타 미용업,9,0.014810,96285.714286,7.985714e+04,99857.142857,3.786820e+06,3.397973e+06,3.524075e+06,761025.0,804979.761905,679950.0


#### 라벨 부착하기

In [27]:
# 현 시점에서 t + 1, t + 2, t + 3의 매출을 부착해야 함

In [28]:
amount_sum_per_t_and_sid.drop(['region', 'type_of_business'], axis = 1, inplace = True)
for k in range(1, 4):
    amount_sum_per_t_and_sid['t_{}'.format(k)] = amount_sum_per_t_and_sid['t'] - k   
    train_df = pd.merge(train_df, amount_sum_per_t_and_sid.drop('t', axis = 1), left_on = ['store_id', 't'], right_on = ['store_id', 't_{}'.format(k)])
    train_df.rename({"amount": "Y_{}".format(k)}, axis = 1, inplace = True)
    
    train_df.drop(['t_{}'.format(k)], axis = 1, inplace = True)
    amount_sum_per_t_and_sid.drop(['t_{}'.format(k)], axis = 1, inplace = True)      

In [29]:
train_df['Y'] = train_df['Y_1'] + train_df['Y_2'] + train_df['Y_3']

## 학습 데이터 탐색 및 전처리

#### 특징과 라벨 분리

In [30]:
X = train_df.drop(['store_id', 'region', 'type_of_business', 't', 'Y_1', 'Y_2', 'Y_3', 'Y'], axis = 1)
Y = train_df['Y']

#### 데이터 분할 및 구조 탐색

In [31]:
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)
Train_X.shape # 특징 대비 샘플이 많음

(37673, 10)

In [32]:
Train_Y.describe()

count    3.767300e+04
mean     3.438967e+06
std      5.193306e+06
min      0.000000e+00
25%      1.135857e+06
50%      2.209400e+06
75%      4.096993e+06
max      1.727659e+08
Name: Y, dtype: float64

#### 이상치 제거

In [34]:
import numpy as np
def IQR_rule(val_list): # 한 특징에 포함된 값 (열 벡터)
    # IQR 계산    
    Q1 = np.quantile(val_list, 0.25)
    Q3 = np.quantile(val_list, 0.75)
    IQR = Q3 - Q1
    
    # IQR rule을 위배하지 않는 bool list 계산 (True: 이상치 X, False: 이상치 O)
    not_outlier_condition = (Q3 + 1.5 * IQR > val_list) & (Q1 - 1.5 * IQR < val_list)
    return not_outlier_condition

In [35]:
Y_condition = IQR_rule(Train_Y)
Train_Y = Train_Y[Y_condition]
Train_X = Train_X[Y_condition]

#### 치우침 제거

In [36]:
# 모두 좌로 치우침을 확인
Train_X.skew()

평균할부율                                  2.999179
1_before_amount                        2.535920
2_before_amount                        2.455859
3_before_amount                        2.528445
1_before_amount_of_region              3.304734
2_before_amount_of_region              3.238923
3_before_amount_of_region              3.224613
1_before_amount_of_type_of_business    1.812173
2_before_amount_of_type_of_business    1.897651
3_before_amount_of_type_of_business    1.920802
dtype: float64

In [37]:
# 치우침 제거
import numpy as np
biased_variables = Train_X.columns[Train_X.skew().abs() > 1.5] # 왜도의 절대값이 1.5 이상인 컬럼만 가져오기
Train_X[biased_variables] = Train_X[biased_variables] - Train_X[biased_variables].min() + 1
Train_X[biased_variables] = np.sqrt(Train_X[biased_variables])

In [38]:
Train_X.skew()

평균할부율                                  2.782873
1_before_amount                        0.739012
2_before_amount                        0.731783
3_before_amount                        0.743576
1_before_amount_of_region              1.874513
2_before_amount_of_region              1.826711
3_before_amount_of_region              1.814483
1_before_amount_of_type_of_business   -0.272532
2_before_amount_of_type_of_business   -0.249009
3_before_amount_of_type_of_business   -0.257339
dtype: float64

#### 스케일링 수행

In [39]:
Train_X.max() - Train_X.min() # 특징 간 스케일 차이가 큼을 확인 => 스케일이 작은 특징은 영향을 거의 주지 못할 것이라 예상됨

평균할부율                                     0.379933
1_before_amount                        3694.924532
2_before_amount                        3595.029696
3_before_amount                        3595.029696
1_before_amount_of_region              2588.236153
2_before_amount_of_region              2588.236153
3_before_amount_of_region              2588.236153
1_before_amount_of_type_of_business    2581.717478
2_before_amount_of_type_of_business    2581.717478
3_before_amount_of_type_of_business    2614.039115
dtype: float64

In [40]:
# 스케일링 수행
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(Train_X)
s_Train_X = scaler.transform(Train_X)
s_Test_X = scaler.transform(Test_X)

Train_X = pd.DataFrame(s_Train_X, columns = Train_X.columns)
Test_X = pd.DataFrame(s_Test_X, columns = Train_X.columns)

del s_Train_X, s_Test_X # 메모리 관리를 위해, 불필요한 값은 제거

#### 모델 학습 

샘플 대비 특징이 적고, 특징의 타입이 전부 연속형으로 같음

- 모델 1. kNN
- 모델 2. RandomForestRegressor
- 모델 3. LightGBM

- 특징 선택: 3 ~ 10개 (기준: f_regression)

In [41]:
from sklearn.model_selection import ParameterGrid
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.ensemble import RandomForestRegressor as RFR
from lightgbm import LGBMRegressor as LGB
from sklearn.feature_selection import *

In [42]:
# 파라미터 그리드 생성
param_grid = dict() 
# 입력: 모델 함수, 출력: 모델의 하이퍼 파라미터 그리드

# 모델별 파라미터 그리드 생성
param_grid_for_knn = ParameterGrid({"n_neighbors": [1, 3, 5, 7],
                           "metric":['euclidean', 'cosine']})

param_grid_for_RFR = ParameterGrid({"max_depth": [1, 2, 3, 4],
                           "n_estimators":[100, 200],
                                   "max_samples":[0.5, 0.6, 0.7, None]}) # 특징 대비 샘플이 많아서 붓스트랩 비율 (max_samples)을 설정 

param_grid_for_LGB = ParameterGrid({"max_depth": [1, 2, 3, 4],
                                   "n_estimators":[100, 200],
                            "learning_rate": [0.05, 0.1, 0.15]})

# 모델 - 하이퍼 파라미터 그리드를 param_grid에 추가
param_grid[KNN] = param_grid_for_knn
param_grid[RFR] = param_grid_for_RFR
param_grid[LGB] = param_grid_for_LGB

In [43]:
# 출력을 위한 max_iter_num 계산
max_iter_num = 0
for k in range(10, 2, -1):
    for M in param_grid.keys():
        for P in param_grid[M]:
            max_iter_num += 1
           
from sklearn.metrics import mean_absolute_error as MAE

best_score = 9999999999
iteration_num = 0
for k in range(10, 2, -1): # 메모리 부담 해소를 위해, 1씩 감소시킴
    selector = SelectKBest(f_regression, k = k).fit(Train_X, Train_Y)
    selected_features = Train_X.columns[selector.get_support()]

    Train_X = Train_X[selected_features]
    Test_X = Test_X[selected_features]
    
    for M in param_grid.keys():
        for P in param_grid[M]:
            # LightGBM에서 DataFrame이 잘 처리되지 않는 것을 방지하기 위해 .values를 사용
            model = M(**P).fit(Train_X.values, Train_Y.values)
            pred_Y = model.predict(Test_X.values)
            score = MAE(Test_Y.values, pred_Y)
            
            if score < best_score:
                best_score = score
                best_model = M
                best_paramter = P
                best_features = selected_features    
                
            iteration_num += 1
            print("iter_num:{}/{}, score: {}, best_score: {}".format(iteration_num, max_iter_num, round(score, 2), round(best_score, 2)))

iter_num:1/512, score: 3021506.84, best_score: 3021506.84
iter_num:2/512, score: 3469749.02, best_score: 3021506.84
iter_num:3/512, score: 3910504.5, best_score: 3021506.84
iter_num:4/512, score: 4117807.56, best_score: 3021506.84
iter_num:5/512, score: 2033985.17, best_score: 2033985.17
iter_num:6/512, score: 1771048.67, best_score: 1771048.67
iter_num:7/512, score: 1700945.73, best_score: 1700945.73
iter_num:8/512, score: 1669355.95, best_score: 1669355.95
iter_num:9/512, score: 3056044.81, best_score: 1669355.95
iter_num:10/512, score: 3053635.91, best_score: 1669355.95
iter_num:11/512, score: 3062137.3, best_score: 1669355.95
iter_num:12/512, score: 3053930.52, best_score: 1669355.95
iter_num:13/512, score: 3059623.4, best_score: 1669355.95
iter_num:14/512, score: 3059730.71, best_score: 1669355.95
iter_num:15/512, score: 3062679.78, best_score: 1669355.95
iter_num:16/512, score: 3057346.77, best_score: 1669355.95
iter_num:17/512, score: 3821982.02, best_score: 1669355.95
iter_num:

iter_num:140/512, score: 3060142.8, best_score: 1629165.78
iter_num:141/512, score: 3050779.16, best_score: 1629165.78
iter_num:142/512, score: 3061130.38, best_score: 1629165.78
iter_num:143/512, score: 3060973.62, best_score: 1629165.78
iter_num:144/512, score: 3055940.34, best_score: 1629165.78
iter_num:145/512, score: 3812950.1, best_score: 1629165.78
iter_num:146/512, score: 3795766.02, best_score: 1629165.78
iter_num:147/512, score: 3810306.56, best_score: 1629165.78
iter_num:148/512, score: 3805655.87, best_score: 1629165.78
iter_num:149/512, score: 3805761.28, best_score: 1629165.78
iter_num:150/512, score: 3802028.65, best_score: 1629165.78
iter_num:151/512, score: 3805862.05, best_score: 1629165.78
iter_num:152/512, score: 3802229.05, best_score: 1629165.78
iter_num:153/512, score: 4248499.24, best_score: 1629165.78
iter_num:154/512, score: 4262598.32, best_score: 1629165.78
iter_num:155/512, score: 4246161.33, best_score: 1629165.78
iter_num:156/512, score: 4257008.27, best_

iter_num:277/512, score: 3797941.83, best_score: 1629165.78
iter_num:278/512, score: 3805065.29, best_score: 1629165.78
iter_num:279/512, score: 3793921.11, best_score: 1629165.78
iter_num:280/512, score: 3797948.1, best_score: 1629165.78
iter_num:281/512, score: 4252787.97, best_score: 1629165.78
iter_num:282/512, score: 4259281.39, best_score: 1629165.78
iter_num:283/512, score: 4247688.14, best_score: 1629165.78
iter_num:284/512, score: 4247496.52, best_score: 1629165.78
iter_num:285/512, score: 4248356.35, best_score: 1629165.78
iter_num:286/512, score: 4236964.13, best_score: 1629165.78
iter_num:287/512, score: 4260229.4, best_score: 1629165.78
iter_num:288/512, score: 4250699.67, best_score: 1629165.78
iter_num:289/512, score: 4483399.9, best_score: 1629165.78
iter_num:290/512, score: 4431402.8, best_score: 1629165.78
iter_num:291/512, score: 4482727.47, best_score: 1629165.78
iter_num:292/512, score: 4416807.74, best_score: 1629165.78
iter_num:293/512, score: 4408335.85, best_sc

iter_num:414/512, score: 4251870.27, best_score: 1629165.78
iter_num:415/512, score: 4260041.43, best_score: 1629165.78
iter_num:416/512, score: 4244276.99, best_score: 1629165.78
iter_num:417/512, score: 4474525.58, best_score: 1629165.78
iter_num:418/512, score: 4497556.34, best_score: 1629165.78
iter_num:419/512, score: 4457162.75, best_score: 1629165.78
iter_num:420/512, score: 4485910.0, best_score: 1629165.78
iter_num:421/512, score: 4493348.09, best_score: 1629165.78
iter_num:422/512, score: 4437960.97, best_score: 1629165.78
iter_num:423/512, score: 4453958.88, best_score: 1629165.78
iter_num:424/512, score: 4478080.47, best_score: 1629165.78
iter_num:425/512, score: 4157997.49, best_score: 1629165.78
iter_num:426/512, score: 4517565.41, best_score: 1629165.78
iter_num:427/512, score: 4488467.69, best_score: 1629165.78
iter_num:428/512, score: 4453144.37, best_score: 1629165.78
iter_num:429/512, score: 4406883.03, best_score: 1629165.78
iter_num:430/512, score: 4511473.63, best

#### 최종 모델 학습

In [44]:
def pipeline(X):
    X[biased_variables] = X[biased_variables] - X[biased_variables].min() + 1
    X[biased_variables] = np.sqrt(X[biased_variables])        
    X = pd.DataFrame(scaler.transform(X), columns = X.columns)
    X = X[best_features]
    return X
    
model = best_model(**best_paramter).fit(pipeline(X).values, Y)

## 적용 데이터 구성

In [45]:
# 2019-03-01 ~ 2019-05-31
submission_df['t'] = (2019 - 2016) * 12 + 2

In [46]:
# region 변수와 type_of_business 변수 부착 
submission_df['region'] = submission_df['store_id'].replace(store_to_region)
submission_df['type_of_business'] = submission_df['store_id'].replace(store_to_type_of_business)

#### 특징 부착

In [47]:
submission_df['평균할부율'] = submission_df['store_id'].replace(installment_term_per_store.to_dict())
submission_df.head()

,store_id,amount,t,region,type_of_business,평균할부율
0,0,0,38,없음,기타 미용업,0.038384
1,1,0,38,없음,없음,0.000000
2,2,0,38,없음,없음,0.083904
3,4,0,38,서울 종로구,없음,0.001201
4,5,0,38,없음,의복 액세서리 및 모조 장신구 도매업,0.075077


In [48]:
submission_df.drop('amount', axis = 1, inplace = True)

In [49]:
# t - k (k = 1, 2, 3) 시점의 부착
# submission_df의 t는 amount_sum_per_t_and_sid의 t-k과 부착되어야 하므로, amount_sum_per_t_and_sid의 t에 k를 더함

for k in range(1, 4):
    amount_sum_per_t_and_sid['t_{}'.format(k)] = amount_sum_per_t_and_sid['t'] + k    
    submission_df = pd.merge(submission_df, amount_sum_per_t_and_sid.drop('t', axis = 1), left_on = ['store_id', 't'], right_on = ['store_id', 't_{}'.format(k)])
    submission_df.rename({"amount":"{}_before_amount".format(k)}, axis = 1, inplace = True)
    submission_df.drop(['t_{}'.format(k)], axis = 1, inplace = True)
    amount_sum_per_t_and_sid.drop(['t_{}'.format(k)], axis = 1, inplace = True)
    


In [50]:
# 지역 관련 변수 부착
for k in range(1, 4):
    amount_mean_per_t_and_region['t_{}'.format(k)] = amount_mean_per_t_and_region['t'] + k
    submission_df = pd.merge(submission_df, amount_mean_per_t_and_region.drop('t', axis = 1), left_on = ['region', 't'], right_on = ['region', 't_{}'.format(k)])
    submission_df.rename({"amount":"{}_before_amount_of_region".format(k)}, axis = 1, inplace = True)
    
    submission_df.drop(['t_{}'.format(k)], axis = 1, inplace = True)
    amount_mean_per_t_and_region.drop(['t_{}'.format(k)], axis = 1, inplace = True)    

In [51]:
# t - k (k = 1, 2, 3) 시점의 부착
# submission_df의 t는 amount_sum_per_t_and_sid의 t-k과 부착되어야 하므로, amount_sum_per_t_and_sid의 t에 k를 더함

for k in range(1, 4):
    amount_mean_per_t_and_type_of_business['t_{}'.format(k)] = amount_mean_per_t_and_type_of_business['t'] + k
    submission_df = pd.merge(submission_df, amount_mean_per_t_and_type_of_business.drop('t', axis = 1), left_on = ['type_of_business', 't'], right_on = ['type_of_business', 't_{}'.format(k)])
    submission_df.rename({"amount":"{}_before_amount_of_type_of_business".format(k)}, axis = 1, inplace = True)
    
    submission_df.drop(['t_{}'.format(k)], axis = 1, inplace = True)
    amount_mean_per_t_and_type_of_business.drop(['t_{}'.format(k)], axis = 1, inplace = True)       
    
submission_df.head()

,store_id,t,region,type_of_business,평균할부율,1_before_amount,2_before_amount,3_before_amount,1_before_amount_of_region,2_before_amount_of_region,3_before_amount_of_region,1_before_amount_of_type_of_business,2_before_amount_of_type_of_business,3_before_amount_of_type_of_business
0,0,38,없음,기타 미용업,0.038384,682857.142857,874571.428571,676000.000000,9.468777e+05,1.000725e+06,9.886195e+05,585125.0,650055.952381,558241.666667
1,792,38,없음,기타 미용업,0.218887,743214.285714,871071.428571,973857.142857,9.468777e+05,1.000725e+06,9.886195e+05,585125.0,650055.952381,558241.666667
2,1828,38,경기 용인시,기타 미용업,0.195502,953000.000000,816857.142857,911957.142857,1.801051e+06,2.009936e+06,1.897275e+06,585125.0,650055.952381,558241.666667
3,23,38,경기 안양시,기타 미용업,0.048795,660857.142857,999285.714286,827571.428571,7.843780e+05,6.421832e+05,6.788446e+05,585125.0,650055.952381,558241.666667
4,192,38,경기 화성시,기타 미용업,0.100542,467571.428571,550571.428571,399142.857143,1.209348e+06,1.125181e+06,1.049587e+06,585125.0,650055.952381,558241.666667


In [52]:
submission_X = submission_df[X.columns]
submission_X = pipeline(submission_X)

pred_Y = model.predict(submission_X)

result = pd.DataFrame({"store_id":submission_df['store_id'].values,
                      "pred_amount":pred_Y})

C:\Users\GilseungAhn\AppData\Roaming\Python\Python36\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [53]:
result.sort_values(by = 'store_id')

,pred_amount,store_id
0,5.168055e+06,0
12,1.315465e+06,1
13,1.564663e+06,2
612,5.263719e+06,4
1187,4.485424e+06,5
615,6.877434e+06,6
14,1.698383e+06,7
1209,4.060090e+06,8
624,2.309178e+06,9
15,1.650262e+06,10
